In [397]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import numpy as np
import pandas as pd
import time
import re

In [2]:
pip install selenium

Note: you may need to restart the kernel to use updated packages.


In [398]:
class utilities:
    
    #def 
    
    def wait(self, seconds):

        s = np.random.normal(0, 1)
        time.sleep(seconds*abs(s))
    
    def get_element_by_path(self, driver, xpath):
        try:
            return driver.find_element_by_xpath(xpath)
        except Exception as e:
            print(e)
            return None

In [399]:
class IndeedItemParser:
    def __init__(self):
        self.driverPath = "C:\\Users\\User\\Documents\\selenium\\driver\\chromedriver.exe"
        self.utilities = utilities();
        
    def _get_title(self, driver):
        try:
            title = driver.find_element_by_xpath("//*[@class='jobsearch-DesktopStickyContainer']//h3")
        except Exception as e:
            print(e)
        return title.text
    
    def _get_name(self, driver):
        
        try:    
            name = driver.find_element_by_xpath("//*[contains(@class,'jobsearch-InlineCompanyRating')]//div[1]")
            return name.text
        except Exception as e:
            print(e)
        
    
    def _get_address(self,driver):
         try:   
            address = driver.find_element_by_xpath("//*[contains(@class,'jobsearch-InlineCompanyRating')]//div[3]")
            if address.text == "-":
                address = driver.find_element_by_xpath("//*[contains(@class,'jobsearch-InlineCompanyRating')]//div[4]")
            return address.text
         except Exception as e:
            address = driver.find_element_by_xpath("//span[@class='jobsearch-JobMetadataHeader-iconLabel'][1]")
            return address.text
    
    def _get_salaire(self,driver, description):
        #jobsearch-JobMetadataHeader-item 
        """
        pattern = r'[[S|s]alaire?[\s+]?:?[\s+]?(.*)?[\s+]?|(.*)?[\s+]?(par|\/)?[\s+]?(an|mois|jour|heure)'
        text = description
        try:
            text = driver.find_element_by_xpath("//*[contains(@class,'jobsearch-JobMetadataHeader-item')]//span[1]").text
            print("first :", text)
            if text == "":
                raise Exception("empty")
        except Exception as e:
            print(e)
            try:   
                text = driver.find_element_by_xpath("//*[contains(@class,'icl-IconFunctional--salary')]//*[contains(@class,'jobsearch-JobMetadataHeader-iconLabel')]").text
                print("second : ",text)
            except Exception as e:
                print(e)
                print("troisième")
                
        outer = re.compile(pattern)
        m = outer.search(text)
        if m is not None:
              return m.group(1)
        return "-"
        """
        return "-"
            
    
    def _get_description(self,driver):
        try:
            #jobDescriptionText
            e_description = driver.find_element_by_id("jobDescriptionText")
            return e_description.text
        except Exception as e:
            print(e)
    
    def _get_date(self,driver,url,name):
       # print(url)
        try:
            date_str = driver.find_element_by_xpath("//*[@class='jobsearch-JobMetadataFooter']")
            date_str_full = date_str.text
            date_tbl = date_str_full.split(" ")
            count_str = date_tbl[4]
            label = date_tbl[5]
            
            if name in date_str_full:
                date_str_full = date_str_full.replace(name, "")
                date_tbl = date_str_full.split(" ")
                count_str = date_tbl[5]
                label = date_tbl[6]
                
            if count_str == "a" :
                count_str = date_tbl[5]
                label = date_tbl[6]
                
            #print("date_str", date_tbl)
            date = datetime.now()
            
            if count_str == "30+":
                return date - timedelta(days=30)
            
            count = int(count_str)
            if "jour" in label:
                date = date - timedelta(days=count)
            elif "heur" in label:
                date = date - timedelta(hours=count)
            return date;
        except Exception as e:
            print(e)
    
    
    def parse(self,url):
        driver = webdriver.Chrome(self.driverPath)
        driver.get(url)
        driver.maximize_window()
        
        source = driver.page_source
        title = self._get_title(driver)
        name = self._get_name(driver)
        address = self._get_address(driver)
        date = self._get_date(driver, url,name)
        description = self._get_description(driver)
        salaire = self._get_salaire(driver,description)
        
        driver.quit()
        
        return title, name, address, date, salaire, description, source

In [400]:
driver = webdriver.Chrome("C:\\Users\\User\\Documents\\selenium\\driver\\chromedriver.exe")
item_link ="https://www.indeed.fr/voir-emploi?cmp=Agence-ULTEAM&t=Chef+%C3%89quipe+Courrier&jk=c3b6cc0a43b192f4&sjdu=QwrRXKrqZ3CNX5W-O9jEvcttsaZdmkbfXA_O1RUuLLVt7V37OcJjVeywo21sCKKVZsKjH1QMOcrJLxLEe3LwbneMTtyfmo-r9mk-65EMnF_xE1dLTd8uf9wEDXC-y3DjRkj44UiUNV_6R9APTWrRDA&tk=1dm9ttu0492gm801&adid=316016867&vjs=3" 
driver.get(item_link)
source = driver.page_source
source_file = 'source.page.{0}.html'.format(item_link.replace("/","_").replace(":","_").replace(".","_").replace("-","_").replace("+","_").replace("?","_").replace("%","_"))
if not os.path.exists("source_file.html"):
    file = open("source_file.html", "w") 
    file.write(source) 
    file.close() 

In [395]:
#item_parser = IndeedItemParser()
#title, name, address, date, salaire,description,source = item_parser.parse("https://www.indeed.fr/voir-emploi?cmp=ECO--SI&t=Technicien+Support+Informatique&jk=dc1a032be2312c84&sjdu=QwrRXKrqZ3CNX5W-O9jEvX5ViC7ZmtCbUlpLMENSqzeEixTsazLVFu1bx1PMxHpgK0V934oVal0W7yYsbyobejEeYX4jvfhEEiUY8zoCrBk&tk=1dm9ttu0492gm801&adid=314524947&vjs=3")
#print(source)

In [401]:
import os.path
from os import path
import json

class IndeedParserStateObject(object):
    def __init__(self, data):
        self.__dict__ = json.loads(data)
        
class IndeedParserStateHandler:
    
    def __init__(self):
        self.filename = "indeed.parser.state.json"
        self.object = self._load_file()
        print(type(self.object))
        #self.object = IndeedParserStateObject(self.data)
    
    def save_state(self, job, location, page_request="", page_item_index=-1):
        self.object["job"] = job
        self.object["location"] = location
        self.object["page_request"] = page_request
        self.object["page_item_index"]  = page_item_index
        #self.data  = json.dumps(self.object)
        
        #data = json.dumps(self.object)
        self._save_file(self.object)
        #self.object = IndeedParserStateObject(self.data)
        
    def is_current_job(self, job):
        if (self.object["job"] == "") | (self.object["job"] == job):
            return True
        return False
    
    def is_current_location(self, location):
        if (self.object["location"] == "") | (self.object["location"] == location):
            return True
        return False
    
    def is_current_page_request(self, page_request):
        if (self.object["page_request"] == "") | (self.object["page_request"] == page_request):
            return True
        return False
    
    def _save_file(self, data):
         with open(self.filename, 'w') as outfile:
                json.dump(data, outfile)
                
    def _load_file(self):
        
        if path.exists(self.filename) == True:
            with open(self.filename) as json_file:
                data = json.load(json_file)
        else:
            data = {
                     "job": "",
                     "location":"",
                     "page_request":"",
                     "page_item_index":-1,
                    }
            with open(self.filename, 'w') as outfile:
                json.dump(data, outfile)
        
        return data


In [420]:
from datetime import datetime, timedelta
import os.path
import os
class IndeedPaser:
    def __init__(self):
        self.website = "https://www.indeed.fr"
        self.driverPath = "C:\\Users\\User\\Documents\\selenium\\driver\\chromedriver.exe"
        #self.dataset = pd.DataFrame(columns=['URL', 'Titre','Nom entreprise','Adresse','Date de publication', 'description'])
        
        if os.path.exists('indeed.csv') == True:
            self.dataset = pd.read_csv("indeed.csv")
        else:
            #URL,Titre,Nom entreprise,Adresse,Date de publication,salaire, description
            self.dataset = pd.DataFrame(columns=['URL','Titre','Nom entreprise','Adresse','Date de publication','salaire','description'])
        
        self.jobs = ["data scientist", "développeur", "data analyst", "business intelligence"]
        self.locations = ["Lyon","Paris", "Toulouse", "Nantes", "Bordeaux"]
        #self.locations = ["Paris", "Lyon", "Toulouse", "Nantes", "Bordeaux"]
        self.utilities =  utilities()
        self.indeed_item_parser = IndeedItemParser()
        self.parser_state = IndeedParserStateHandler()
    
    def _get_pages_counts(self,driver):
        #searchCountPages
        searchCountPages_elt = driver.find_element_by_id("searchCountPages")
        searchCountPages = searchCountPages_elt.text.split()
        if len(searchCountPages) == 6:
            searchCountPages = int("{0}{1}".format(searchCountPages[3],searchCountPages[4])) 
        else :
            searchCountPages = searchCountPages[3]  
        
        return (int(searchCountPages) // 18)
    
    def parse(self):
        browser = webdriver.Chrome(self.driverPath)
        browser.get(self.website)
        browser.maximize_window()
        
        for job in self.jobs:
            
            if self.parser_state.is_current_job(job) == False:
                continue
                    
            for location in self.locations:
                
                if self.parser_state.is_current_location(location) == False:
                    continue
                self.parser_state.save_state(job, location)
                
                query = "https://www.indeed.fr/jobs?q={0}&l={1}".format(job, location)
                browser.get(query)
                    
                pages_count = self._get_pages_counts(browser)
                
                for page_index in range(1, pages_count):
                    full_query = "{0}&start={1}".format(query,page_index)
                    
                    browser.get(full_query)
                    
                    items = browser.find_elements_by_xpath("//*[contains(@class,'clickcard')]")
                    dataset_len = len(self.dataset)
                    
                    for index_i,item in enumerate(items): 
                        title = item.find_element_by_xpath(".//*[contains(@class,'jobtitle')]")
                        item_link = title.get_attribute("href")
                        
                        try:
                            if (self.dataset["URL"].isin([item_link]).any()):
                                print(item_link)
                            else:
                                title, name, address, date,salaire, description, source = self.indeed_item_parser.parse(item_link)
                                self.dataset.loc[dataset_len + index_i] = [item_link, title, name, address, date, salaire, description]
                                
                                #archivage des page html
                                source_file_name = "source_file.{0}.html".format(dataset_len + index_i)
                                if not os.path.exists(source_file_name):
                                    file = open("pages/{0}".format(source_file_name), "w")
                                    source = item_link + "----------------" + source 
                                    file.write(source) 
                                    file.close() 
                                    
                                print("len(dataset)", len(self.dataset))
                                
                        except Exception as e:
                            print(e)
                            #if (len(self.dataset) == 0) or (True not in self.dataset["URL"].isin([item_link])):
                        
                        #break
                    #break
                    self.dataset.to_csv("indeed.csv",index=False)
                    #https://www.indeed.fr/jobs?q=developpeur&l=paris&start=10
                    
                print(searchCountPages)
                #break
            #break    
            self.dataset.drop_duplicates(subset ="URL", keep = False, inplace = True) 

In [ ]:
test = IndeedPaser()
test.parse()

<class 'dict'>
chelou 39
len(dataset) 1459
len(dataset) 1460
len(dataset) 1461
len(dataset) 1462
len(dataset) 1463
len(dataset) 1464
len(dataset) 1465


In [10]:
pip install pymongo

Note: you may need to restart the kernel to use updated packages.


In [12]:
pip install django

Note: you may need to restart the kernel to use updated packages.


In [403]:
class CsvToMongodbMigratorTool:
    
    def __init__(self):
        self.dataset = pd.read_csv("indeed.csv")
        self.mongodbdao = IndeedMongodbDao()
        
    def migrate(self):
        records = json.loads(self.dataset.T.to_json()).values()
        self.mongodbdao.insert_data_bulk(records)
        

In [409]:
from selenium import webdriver
import bs4
from bs4 import BeautifulSoup

data_set = pd.read_csv("indeed.csv")
#data_set
#data_set
#for index, row in data_set.iterrows():
#    xtraSoup = BeautifulSoup(row["description"])
#    body = xtraSoup.find('body')
#    row["description"] = body.text
    
#data_set.to_csv("indeed.csv",index=False)
        
#data_set = data_set.drop(columns=['index'])
#data_set
#print(len(data_set))
#data_set.drop_duplicates(subset ="URL", keep = False, inplace = True) 
#print(len(data_set))
#data_set.to_csv("indeed.csv",index=False)
#record = data_set.iloc[1]

#indeeddao = IndeedMongodbDao()
#indeeddao.insert_data(record["URL"],record["Titre"], record["Nom entreprise"],record["Adresse"], record["Date de publication"], record["description"])


In [67]:
data_set["URL"].iloc[10]

'https://www.indeed.fr/company/LEC-Conseil/jobs/D%C3%A9veloppeur-Fullstack-F-H-ad4f8380f4e66e4a?fccid=3d3e7ac112fae637&vjs=3'

In [82]:
#migratorTool = CsvToMongodbMigratorTool()
#migratorTool.migrate()

In [407]:
from pymongo import MongoClient 
from pymongo import errors 
from django.core.validators import URLValidator
from django.core.exceptions import ValidationError
class IndeedMongodbDao:
    def __init__(self):
        self.conn = MongoClient() 
        self.db = self.conn.Indeed
        self.collection = self.db.data
        self._create_index()
    
    def _create_index(self):
        index_name = 'url'
        if index_name not in self.collection.index_information():
            self.collection.create_index(index_name, unique=True)
    
    def _valid_url_format(self,url):
        val = URLValidator()
        try:
            val(url)
        except ValidationError as e:
            raise Exception('bad format for url {}'.format(ur))
    
    def insert_data_bulk(self,data):
        self.collection.insert_many(data)
    
    def insert_data(self, url, title, name, address, publication_date, description):
        
        try:
            if url == "":
                raise Exception('url cannot be empty {}'.format(ur))

            self._valid_url_format(url)

            if title == "":
                raise Exception('title cannot be empty {}'.format(title))

            if name == "":
                raise Exception('the name of company cannot be be empty {}'.format(title))

            if description == "":
                raise Exception('description of company cannot be be empty {}'.format(title))


            line_to_insert = {
                                "URL": url,
                                "title":title,
                                "name":name,
                                "adresse":address,
                                "publication_date":publication_date,
                                "description":description
                             }

            # Insert Data 
            result = self.collection.insert_one(line_to_insert) 
        except errors.DuplicateKeyError as e:
            print('DuplicateKeyError : url {0} aready exist'.format(url))
    
    def get_all_data(self):
        data = self.collection.find({})
        return data
    
    

In [106]:
import numpy as np 
class IndedRegexParser:
    
    def __init__(self, descriptif_text ):
        self.descriptif_text = descriptif_text
    
    def _get_items(self,pattern):   
        result_re = re.findall(pattern,self.descriptif_text)
        result = []
        for item_l_1 in result_re:
            for item_l_2 in item_l_1:
                if item_l_2 != "":
                    result.append(item_l_2)
        return result
    
    def _get_niveau_experience(self):
        #Niveau d'expérience : regex true
        result = self._get_items('([(e|E)\w+]xpérience?:?\s\d{1,15} ans?)|([(e|E)\w+]xpérience?\s?de\s?\d{1,15} ans?)|(\d{1,15} ans d\'[e|E]xpérience[s]?)')
        if len(result) == 0:
            outer = re.compile(r'Expérience:\s*.+:( (\w) (an|ans) \(((R|r)equis|(s|S)ouhaité)\))')
            m = outer.search(self.descriptif_text)
            if m is not None:
                return m.group(1)
            return []
    
    def _get_salaires(self):
        #Salaire ajusté en fonction du coût de la vie dans les différentes villes : regex true
        #return self._get_items('(\d{2,3}\s?(k|K)?)|(\d{2,3}\s?\d{3,5}\s?(k|K)?)|([(s|S)]alaire?[\s]?:?[\s]?\d{2,6}k?)|([(r|R)]émunération?[\s]?:?[\s]?\d{2,6}k?)|([(g|G)]ratification?[\s]?:?[\s]?\d{2,6}k?)') 
        #[[S|s]alaire?[\s+]?:?[\s+]?(.*)e?[\s+]?\/an
        outer = re.compile(r'[[S|s]alaire?[\s+]?:?[\s+]?(.*)e?[\s+]?\/(an|mois)|(.*)?[\s+]?par?[\s+]?mois')
        m = outer.search(self.descriptif_text)
        if m is not None:
            return m.group(1)
        return []
            
        
    
    def get_langages_prog(self):
        #Langages de programmation requis (R, Python, Excel (VBA), SQL, C++, Java, SAS...): regex true
        return self._get_items('(R+[^\w])|([(p|P)\w+]ython?)|(Excel?)|(VBA?)|(C\+\+?)||(C?)|(Asp.net?)|(SQL?)|(NoSQL?)|(Linux?)|(MySQL?)|(MongoDB?)|(DBMaria?)|(SQL Server?)||(Java?)|(JavaScript?)|(PHP?\s?\d{1,}?)|(HTML?\s?\d{1,}?)|(CSS?\s?\d{1,}?)|(SAS?)|(C#?)|(Ruby?)|(Swift?)|(Objective-C?)|(VB.NET?)|(Kotlin?)|(Scala?)|(Bash?)|(PowerShell?)|(Shell?)|(Front End?)|(Back End?)')
    
    def get_outils_re(self):
        #Outils requis (Tableau, PowerBI...) : regex true
        return self._get_items('([(t|T)\w+]ableau?)|([(p|P)\w+]ower[(b|B)\w+][(i|I)\w+]?)|([(s|S)\w+]ymfony?\s?\d{1,}?)|([(j|J)\w+][(q|Q)\w+]uery?\s?\d{1,}?)|([(a|A)\w+]ngular?[\s?\d{1,}]?)|([(r|R)\w+]eact [(j|J)\w+][(s|S)\w+]?)|(React Native?)|(Node JS?)|([(g|G)\w+]it?)|(Visual Studio?)|(Visual Studio Code?)|(Django?)|(Flask?)|(API REST?)|(http?s?)|(HTTP?S?)')
        
        
    def get_niv_etudes_re(self):
        #Niveau d'études requis : regex true
        return self._get_items('([(f|F)\w+]ormation?\s?[(b|B)\w+]ac\s\+\s[1-8])|([(b|B)\w+]ac\s\+\s[1-8])')
        
    def get_type_cursus_re(self):
        #Type de cursus (école ingénieur, master, autodidacte...) : regex true
        return self._get_items('([(é|E)\w+]cole [(i|I)\w+]ngénieur?)|([(m|M)\w+]aster?\s?\w{3,25})|([(a|A)\w+]utodidacte?)|([(g|G)\w+]rande[s]? [(é|E)\w+]cole[s]?)|([(é|E)\w+]cole[s]? de [(c|C)\w+]ommerce[s]?)|([(i|I)\w+]ngénieur [(i|I)\w+]nformatique?)')
        
    def get_type_contrat_re(self):
        #Type de contrat : regex true
        return self._get_items('[(c|C)\w+]ontrat?:?\s\w{3,25}')
    
    def get_categorie_developpeur_re(self):
        #Grande catégorie : développeur (developeur web, dev mobile,dev) / data regex true
        return self._get_items('([(d|D)\w+]éveloppeu[(r|s)]e? [(f|F)\w+]ull? [(s|S)\w+]tack)|([(d|D)\w+]éveloppeu[(r|s)]e? [(f|F)\w+]ront[(\s|\-)][(e|E)\w+]nd?)|(développeu[(r|s)]e? Back[(\s|\-)][(e|E)\w+]nd?)|([(d|D)\w+]éveloppeu[(r|s)]e? [(m|M)\w+]obile?)|([(d|D)\w+]éveloppeu[(r|s)]e? [(l|L)\w+]ogiciel?)|([(s|S)\w+]oftware [(i|I)\w+]ngenieer?)|([(d|D)\w+]éveloppeu[(r|s)]e? [(l|L)\w+]ogiciels?)|([(d|D)\w+]éveloppeu[(r|s)]e? [(C|c)\w+]\+\+?)|([(d|D)\w+]éveloppeu[(r|s)]e? [(c|C)\w+]#?)|([(d|D)\w+]éveloppeu[(r|s)]e? [(j|J)\w+]ava?)')
    
    def get_categorie_data_re(self):
        #Grande catégorie : data regex true
        return self._get_items('([(d|D)\w+]ata [(a|A)\w+]nalyst?)|([(d|D)\w+]ata [(s|S)\w+]cientist?)|([(i|I)\w+]ngénieu[(r|s)]e? [(m|M)\w+]achine [(i|I)\w+]earning?)|([(c|C)\w+]onsultant [(b|B)\w+]usiness [(i|I)\w+]ntelligence)|([(c|C)\w+]onsultant [(d|D)\w+]ata)|([(c|C)\w+]onsultant [(b|B)\w+]ig [(d|D)\w+]ata)|([(c|C)\w+]onsultant BI)')
        

In [283]:

dataset = pd.read_csv("indeed.csv")
print(len(dataset))
#for index, row in dataset.head(1000).iterrows():
    #print(row["URL"])
    #indedRegexParser = IndedRegexParser(row["description"])
    #print(indedRegexParser._get_niveau_experience())
    #print(indedRegexParser._get_salaires())
    #print(indedRegexParser.get_outils_re())
    #print(indedRegexParser.get_niv_etudes_re())
    #print(indedRegexParser.get_type_cursus_re())
    #print(indedRegexParser.get_type_contrat_re())
    #print(indedRegexParser.get_categorie_developpeur_re())
    #print(indedRegexParser.get_categorie_data_re())


1458


In [248]:
dataset = pd.read_csv("indeed.csv")
dataset.columns

Index(['URL', 'Titre', 'Nom entreprise', 'Adresse', 'Date de publication',
       'salaire', 'description'],
      dtype='object')

In [392]:
import os
class LocalIndeedPaser:
    def __init__(self):
        self.pages_path = 'C:\\Users\\User\\Documents\\Projet_ML_gr3\\scrapping\pages'
        self.dataset = pd.read_csv("indeed.csv")
        self.salary_pattern = "[[S|s]alaire?[\s+]?:?[\s+]?(.*)e?[\s+]?\/(an|mois)|((.*)?[\s+]?par?[\s+]?(an|ans|mois|jour|heure))"
    
    def _get_salary(self,select_result):
        salary = ""
        for item in select_result:
            if "€" in item.text:
                outer_salary = re.compile(self.salary_pattern)
                m_salary = outer_salary.search(item.text)
                if m_salary is not None:
                    salary = m_salary.group(0)
                    break
        return salary
    
    def save_file(self):
        self.dataset.to_csv("indeed.csv",index=False)
    
    def parse(self):
        index = 0
        for root, dirs, files in os.walk(os.path.abspath(self.pages_path)):
            for file in files:
                local_file = os.path.join(root, file)
                file = open(local_file, "r") 
                html = file.read() 
                outer = re.compile(r'.*\-\-\-\-\-\-\-\-\-\-\-\-\-\-\-\-')
                m = outer.search(html)
                if m is not None:
                    link = m.group(0).replace("----------------","")
                    dataset_index = self.dataset[self.dataset["URL"] == link].index.values.astype(int)
                    #print(self.dataset["salaire"][dataset_index].values[0])
                    salary = self.dataset["salaire"][dataset_index]
                    
                    if (len(salary.values) == 0):
                        continue 
                        
                    if (salary.values[0] != "-"):
                        continue
                        
                    html = html.replace(m.group(0),"")
                    soup = BeautifulSoup(html, "html.parser")
                    result = soup.select(".jobsearch-JobMetadataHeader-item")
                    if len(result) > 0:
                        self.dataset["salaire"][dataset_index] = self._get_salary(result)
                        continue
                        
                    result = soup.select(".jobsearch-JobMetadataHeader-itemWithIcon .jobsearch-JobMetadataHeader-iconLabel")
                    if len(result) > 0:
                        self.dataset["salaire"][dataset_index] = self._get_salary(result)
                        continue
                    
                    result = soup.select("#jobDescriptionText")
                    if len(result) > 0:
                        index = index + 1                          
                        m_salary = outer_salary.search(result[0].text)
                        if m_salary is not None:
                            salary = m_salary.group(0)
                            print(salary)
                            break
                    
                    
                    

In [393]:
localIndeedPaser = LocalIndeedPaser()
localIndeedPaser.parse()

1
https://www.indeed.fr/rc/clk?jk=9cd39c0aa33ef01d&fccid=5f6143d2f338a75f&vjs=3
Nous recherchons pour l’un de nos clients, dans le cadre d’un CDI un Ingénieur Développeur Informatique H/F. Le poste est basé dans le 12ème arrondissement.
MISSIONS / TACHES
Missions principales :
Vous définissez et analysez les besoins des clients
Vous prenez en charge la conception, la réalisation et le suivi de l’usage des composants logiciels
Vous analysez et rédigez les spécifications? Vous suivez, corrigez des bugs, assurez les tests et les mises en production
Vous êtes le garant du bon fonctionnement des logiciels
Vous êtes force de propositions dans la mise en place de nouvelles solutions
Vous assurez une veille technologique
COMPETENCES TECHNIQUES ET/OU FONCTIONNELLES EXIGEES
Vous connaissez les langages C, C++ et/ou Java. Vous êtes capable de passer d’un langage à un autre facilement. Vous disposez d’une forte capacité de travail.Vous disposez d’un goût prononcé pour les mathématiques et la progr

5
https://www.indeed.fr/pagead/clk?mo=r&ad=-6NYlbfkN0Daqc6iXyKFbG6VVD0ldd2zgLQxuS1gW8w6A9JOLP9whFvg-KHK1Ro4tEosOO9riQHIFYafVomZqqnmjALrIyx_4LK8mUJNsU9I7eX6-ARFIk8cfbb4X2hB-8xLtdclgxGtm-m4V5I4CwBDkgif8asVEbO0wCXlQHm73QdSvy-pX66v-DA5NTTtI9GivK3Dd50Xj8KCLWDkLbuvs6xFBLJAIl7Uwcwtc00rrORNiZJCrQO37dZH265crvM3mpLX2GHPPw9LfJQix6ZxoKp55MVODrkYj5e-eXR0HpzToNMOh6K0Y_whMs6wVQ7z1I7H-DBSRqOFIGlt4c-Caon2fQZk7sd8liRyV-Gesx8fX7ZxL8yh0f3MZYcAVI8E5uqoSjpqHeWp0kF6f25AsQUjIWhg8RHWMSUsantRc1YZEmTzszW31AWxPPkOlnbMxFjoSA8=&p=0&fvj=0&vjs=3
Contexte
La programmation est aujourd’hui au cœur de l’innovation et le développeur au centre de l’organisation. Le code ? Il évolue, vite, très vite, au gré des versions et des nouveaux langages. Il ne suffit plus qu’il fonctionne, il faut qu’il soit « clean » et compréhensible par tous. Il est désormais produit par un artisan expert qui est également un relai de connaissances au sein de sa communauté, celle des software craftsmen.

Chez NOVENCIA, même si chacun dispose de c

https://www.indeed.fr/rc/clk?jk=328290eb94e6aee7&fccid=237908de095b6446&vjs=3
Job Description

Le poste est à pourvoir en CDI dans la DSI du Siège International d’Ubisoft à Paris, et plus particulièrement au sein de l’équipe « EBS » dont le rôle est de créer les meilleurs outils pour les employés d’Ubisoft.
Vous travailleriez en binôme avec un autre développeur (qui a un profil plutôt backend) sur des projets de création d’applications et de services web en lien avec la Data.
Concrètement, vous seriez en relation avec une équipe de Data Scientists et de Data Analysts qui ont des besoins en termes d’outils ou d’applications maison : Ils fournissent l’algorithme, vous fournissez le design, l’UX et le développement de l’application.
L’objectif est de créer des outils les plus efficaces possibles pour permettre à nos spécialistes Data de réaliser des analyses poussées et des recommandations prédictives.
À ce titre, vos missions seront les suivantes :
Échanger avec les équipes Data afin de 

15
https://www.indeed.fr/rc/clk?jk=6ca18147b33cc6f7&fccid=4382a21cdec2a600&vjs=3
TA MISSION ?

En tant que développeur Full Stack JS, tu réalises des applications web innovantes sur des projets d’envergure où la performance et l’optimisation sont des données clés.

Tu interviens aussi bien sur la partie Front que Back. Tu contribues à la qualité du développement et tu réalises les tests unitaires.

TON ENVIRONNEMENT ?

Au sein d’une équipe de développeurs, accompagné d’un Chef de Projet et d’un Product Owner, tu travailles en mode Agile.

TES COMPÉTENCES TECHNIQUES ?

Javascript / ReactJS / Node.js / Jest / GraphQL / NoSQL / Git
16
https://www.indeed.fr/pagead/clk?mo=r&ad=-6NYlbfkN0CfNWPNVmY518qXJCXgKr8LztMrbmrQnLSX_v0Y3KfTQ82zFLWeBL5d8aCVfbtc-ormpMIxS9ZiFB1rE0-yXIwg2YwwP76OiXqqkntH-SpOZleAyevAvA_CuIVGWMhlwbX3OEdh57sEpUHOcJy1gE_00j0ZCWpBtY5SmcATO6DV8kFy0f_prryRHvldH3Y2BqVbnDeBbFc18zS_D9KpGtR9JdKLe5hutV4gR0_DAzn361nmUmdMZSSLU_R0Tebw9DI5-3d861kt-WWd_935_fiBk7VO1eGXsB67wLlxZ24A1f3gtR1mbu_

22
https://www.indeed.fr/rc/clk?jk=0df5f9cd957a17f1&fccid=2d0740e02e80cf22&vjs=3
Airbus Defence and Space SAS
Airbus is a global leader in aeronautics, space and related services. In 2018 it generated revenues of € 64 billion and employed a workforce of around 134,000. Airbus offers the most comprehensive range of passenger airliners. Airbus is also a European leader providing tanker, combat, transport and mission aircraft, as well as one of the world's leading space companies. In helicopters, Airbus provides the most efficient civil and military rotorcraft solutions worldwide.

Our people work with passion and determination to make the world a more connected, safer and smarter place. Taking pride in our work, we draw on each other's expertise and experience to achieve excellence. Our diversity and teamwork culture propel us to accomplish the extraordinary - on the ground, in the sky and in space.
Job Description
Un poste de Développeur logiciel IA pour réseaux IP (h/f) vient de s'ouvr

...

91
https://www.indeed.fr/rc/clk?jk=acfbb0d492cd8cc7&fccid=c56ae1a209cde5cf&vjs=3
Emploi

Be Free Talent, le connecteur de talents 2.0

Drupal est un CMS (système de gestion de contenu) semblable à WordPress et Prestashop.
Cependant, c’est un CMS qui offre des possibilités clés en main à toute personne souhaitant créer un site internet très complexe.
Par complexité, nous parlons de tous les cas de figure (technique) qui ne sont pas prises en charge par d’autres CMS. C’est pourquoi, ce logiciel est très utilisé dans les grands groupes. D’ailleurs, les développeurs Drupal H/F maîtrisent très souvent, un ou plusieurs langages de programmation tels que Java et PHP.

En effet, il est possible de concevoir et de développer une architecture parfaite de votre site internet via Drupal en intégrant des protocoles de test.

Enfin, nous avons de nombreux projets via Drupal qui vous attendent : E-commerce, refonte, création site web.

Rejoignez-nous Développeurs seniors Drupal H/F !
92
https://www.i

99
https://www.indeed.fr/pagead/clk?mo=r&ad=-6NYlbfkN0Daqc6iXyKFbG6VVD0ldd2zgLQxuS1gW8w6A9JOLP9whFvg-KHK1Ro4tEosOO9riQG3d4ZpcdpnMeGAzF1m_HpP1tK-Ij6oFTnu0X2kpqSUjIUt5J_eD4sD2JUWM7RD4vFI8IgYd5s4KjQ6tr-wy7oLupn5nM9QXLq4BK_aCRm76yHTk3QQuAKC7Vb2wX7me7ZW6fM9Kln5VtnDVkCrG3rLQucQwYy4D6-VGHSh2FHGx_RhbfvlWOJ5MT5OHxZlNlOPmGdqlGimiXGNgDNH-RxH64rHJI8ggvjwaHB3mpQ6Kf1TUfvK4JsEEJr3YH1nlWDNPcykjlwmVDO6hiwRjM6PKzwf_Dlvy6wR-0U6dffVbvonvCPGOeGId8IWrkTO6zTwIgAT_zxhJUvv2NVbpmMzCSzU3XKvxDDlieVkXpoqHvbS8IbIOyS0_-9hgxcAGLc=&p=0&fvj=0&vjs=3
Contexte
La programmation est aujourd’hui au cœur de l’innovation et le développeur au centre de l’organisation. Le code ? Il évolue, vite, très vite, au gré des versions et des nouveaux langages. Il ne suffit plus qu’il fonctionne, il faut qu’il soit « clean » et compréhensible par tous. Il est désormais produit par un artisan expert qui est également un relai de connaissances au sein de sa communauté, celle des software craftsmen.

Chez NOVENCIA, même si chacun dispose de 

105
https://www.indeed.fr/rc/clk?jk=8ee618fc0da4253b&fccid=2c1ebe710125ee82&vjs=3
Summary
Fondée en 1917 par l’Espagnol Cristóbal Balenciaga et établie à Paris depuis 1937, la célèbre Maison de couture française a défini le concept de modernité et d’élégance grâce à la maîtrise des techniques et à l’utilisation de tissus innovants.

Demna Gvasalia, nommé directeur artistique des collections en 2015, continue d’honorer la vision de Cristóbal Balenciaga au travers de techniques raffinées et de coupes magistrales qui repoussent constamment les frontières.

Le prêt-à-porter féminin et masculin, la maroquinerie, les chaussures, les accessoires, les lunettes et les parfums pour femmes composent l’offre de Balenciaga.
Job Description
BALENCIAGA - Coordinateur/trice Développement Chaussures
En qualité de Coordinateur/trice Développement Chaussures, le Salarié sera chargé des tâches et des missions suivantes :
Consolidation des différents composants des modèles (matières et/ou informations tech

109
https://www.indeed.fr/pagead/clk?mo=r&ad=-6NYlbfkN0CJzjc_DeZJgfcxYtp403lwLZa7SzuiADXepM-CUMsOofmpTp7BVSIbaWQF6Eg3a67Ku2GjEM8zZSZwcVaOrSNBLepkxy0Dsi8tzKaClVq1dxkD6Zryll6VcxmToVijeyFc1J8fC-18fVuHmRaSK6xBNzf4jDgZIPedocIo4SvlvbEh0o8kc2irMZz7p-3ZsFOQDpSr8O4QuGeKlLCvpQuPmoWzCWfAQJ-X7zDMPZY3HtHXn1MqO8exoBY7K9zr0FXupDWVNEEFGfStdrpOO0pF5PXiID_QNtxN-NtEk0amt9IwUrldCPdymy0PlsWRomCV7GhnObwtQAOH8zkht37nLLrVCmVC5daG1kpentsDjFWxICVJSahkmK9kDHrreGMz4HbOqb0G99orn9G_aMr3HA4RwqzwaxZndOomqzQ6PbD2kIIG0dQo9YDEC5aOE4lAyw5NOl2kqdJvfk-RLoRZUcX1TPf7f5B7YO_NZlvKT1XPZ4doWml5&p=0&fvj=0&vjs=3
Vos Challenges :
Vous rejoignez la communauté API Metro France et évoluez sur des projets en lien avec nos activités de construction d’applications Web ou d’apps, menées en méthodes Agiles pour nos clients et nos collaborateurs.
Vous êtes intégré au sein d’une équipe de développement et placé directement sous la responsabilité du chef de service EAM, local product and Testing.
Vos missions sont les suivantes :
 Mise en pla

117
https://www.indeed.fr/pagead/clk?mo=r&ad=-6NYlbfkN0ChROIijibjfJ25KaVfYIfi5WdnK6HQhPaVnY17Bes_DapPMWLGudjOFycH4PcwuzyeuH6WLvTbchvePywZUkkuA400VSZyPGtp91_8TYfCFFrkMqpLNJXIwubB0HP1c1mJLMHC9wXzCjZvlKEnw_4jLzDMme5uNirkz1jsv84z8mnNkVX_6gohphZDvBCp_ag4-6q_po5Rlh_MSPiRVRw6MIKDSXBvmrnpxY0Dh3tnP7pmdNAqYkPYWg2lDYG43y0SfiHk4QPDXh2u3ji1AqW3iDbUJrG3ao0lREwL6S8qJnxkV62_58vqJvz1zz4Iz0mI1tdYTXOX95LaJu9adkd4v6d0-TWza8VceS65y72oRG_Ax0T56hSVC4HSyE33MFu1IHhFMt6sabdx-_7E-4Akfn2IwtnBZkkJnIis3bl-jDsQXDhlDLkvB-HOYLPxdJsZ83kk8ze3Nyx8gB0Zaf3VGarZ8iTWgyBXXNjy4QH3Zo07XKFnVnwR&p=0&fvj=0&vjs=3
Au sein du Pôle Droit et Réglementation, rattaché(e) au Directeur de la Gestion des Produits Online, le développeur NLP aura pour mission d'accompagner les équipes de développement et produits dans la l'élaboration des nouvelles fonctionnalités et services digitaux de l'entreprise.
À ce titre, il/elle sera amené(e) à travailler non seulement sur notre base de contenu mais également sur des sources d'information autres afin

In [334]:
localIndeedPaser.save_file()

In [349]:
xx = pd.read_csv("indeed.csv")
xx.iloc[5]["URL"]

'https://www.indeed.fr/pagead/clk?mo=r&ad=-6NYlbfkN0Afb2xqmmvnLeU-eGdBKKKG4nWVW1VrA-McngqAoir6deY3vbRrCKR3mm24AFrxzAqh7I0b4RySssf7nCnyJKwdLxN7QMXQhkNsCK8XusozsvmW1eF3sglEZHhdIQHVv2RHiak76E1rzsanbeeCr-cV9qVWesmcumaHAoMbOeEA4CHTqraW36OrylaxzxDaGwF_dfKkQLBx0PCe4_Ff_lSISpdlsfcx6xvq4BZ77T4OvqwxWuJAMl11doXmHjNHuVfMkBP1COQrS9rugwU9E7eEdahFZj9oBzpewwcOVsBIqbEAhkX0Ys22r01XhfaozrQVMtXwdb567ld5AqNVyikTl10F-fkscayOe1CvGQ4IlwT_rWjXaG_9AuaL5eSevzwbi83GyKGC7uU0NkdsS28ts1fuPjxBL2JPI44-dRVWbi3rjwIsExHL3ixrlh9Zsgs=&p=5&fvj=1&vjs=3'

In [346]:
xx

,URL,Titre,Nom entreprise,Adresse,Date de publication,salaire,description
0,https://www.indeed.fr/pagead/clk?mo=r&ad=-6NYl...,Sup3r Développeur Web Full Stack @Startup,GlobeSailor SAS,Paris (75),2019-10-03 03:11:00.197236,NaN,4pp31 4 T0u5 135 D3v310p3r5 w3b full st4k !\nP...
1,https://www.indeed.fr/pagead/clk?mo=r&ad=-6NYl...,Développeur Informatique H/F,VALENTE SECURYSTAR,94320 Thiais,2019-10-02 03:11:14.353355,1 850 € par mois,"VALENTE SECURYSTAR, fabricant de portes blindé..."
2,https://www.indeed.fr/pagead/clk?mo=r&ad=-6NYl...,Développeur Android (H/F),Universign,Paris (75),2019-09-04 03:11:27.090785,NaN,Universign est le pure player de référence sur...
3,https://www.indeed.fr/pagead/clk?mo=r&ad=-6NYl...,Développeur Ionic / Firebase,NEXTT,92200 Neuilly-sur-Seine,2019-09-19 03:11:39.836685,NaN,Dans le cadre du développement d'une applicati...
4,https://www.indeed.fr/pagead/clk?mo=r&ad=-6NYl...,Data Engineer / Développeur NLP H/F,Wolters Kluwer,Paris (75),2019-09-04 03:11:53.014114,NaN,"Au sein du Pôle Droit et Réglementation, ratta..."
5,https://www.indeed.fr/pagead/clk?mo=r&ad=-6NYl...,Développeur Java H/F,Ratp Smart Systems,Paris (75),2019-09-10 03:12:05.948019,NaN,1er jet :\nPour accélérer sa transformation et...
6,https://www.indeed.fr/rc/clk?jk=6986aa9ce3ca25...,Développeur iOS (H/F),Universign,Paris (75),2019-10-01 03:12:18.742210,-,Universign est le pure player de référence sur...
7,https://www.indeed.fr/rc/clk?jk=8ee618fc0da425...,BALENCIAGA - Coordinateur/trice Développement ...,Groupe Kering,Paris (75),2019-10-03 16:12:31.688147,-,Summary\nFondée en 1917 par l’Espagnol Cristób...
8,https://www.indeed.fr/rc/clk?jk=2f40c0f1d86b7b...,Développeur Junior (ReactJS / Node.js),AKOYA,Paris (75),2019-10-01 03:12:44.296664,NaN,AKOYA GENIUS\nEn complément de ses activités d...
9,https://www.indeed.fr/rc/clk?jk=fdb86ccd993a5c...,BALENCIAGA - Coordinateur/trice Développement ...,Balenciaga,Paris (75),2019-10-03 15:12:57.009145,-,Summary\nFondée en 1917 par l’Espagnol Cristób...
